## Retrieve Daylight Data and Begin Processing:

First, add the 'lib' directory of the project in order to import our custom libraries:

In [5]:
import sys, os
project_dir, current = os.path.split(sys.path[0])
lib_dir = os.path.join(project_dir, "lib")
sys.path.insert(0, lib_dir)

import pandas as pd
import getpass

Set up our connection parameters:

In [6]:
hostname = getpass.getpass(prompt='Hostname:')
database = getpass.getpass(prompt='Database name:')
username = getpass.getpass(prompt='Username:')
password = getpass.getpass(prompt='Password:')

Connect to the database

In [7]:
lib_database.database.connect(hostname, database, username, password)

NameError: name 'lib_database' is not defined

Define the Database Parameters

In [5]:
schema = 'valueofdaylight'
engine = "postgresql://" + username + ":" + password + "@" + hostname + "/" + database

Retrieve Data

In [6]:
table = 
#table = lib_database.database.retrieve_table('data', engine, schema)
table

,index,contract_id,retail_notes,tenant_brokers,tenant_brokerage_firms,tenant_industry,landlord_brokers,landlord_brokerage_firms,landlord_name,parking_notes,...,inter_sDA3_flr16,inter_sDA3_flr31,inter_sDA3_flr46,inter_sDA4_flr0,inter_sDA4_flr16,inter_sDA4_flr31,inter_sDA4_flr46,sDA_leed_0_55,Unnamed: 452,Unnamed: 453
0,0,243552566,None,"Brian Goldman, Eric Cagner",Newmark Grubb Knight Frank,Apparel/Textile,"Ken Rapp, Peter Turchin, Adam Foster, Mike Riz...","CBRE, Brookfield Properties","Brookfield Asset Management, China Investment ...",NaN,...,0,0,0,0,0,0,0,0,0,1
1,1,243552566,None,"Brian Goldman, Eric Cagner",Newmark Grubb Knight Frank,Apparel/Textile,"Ken Rapp, Peter Turchin, Adam Foster, Mike Riz...","CBRE, Brookfield Properties","Brookfield Asset Management, China Investment ...",NaN,...,0,0,0,0,0,0,0,0,0,1
2,2,243555525,None,Zarba,CBRE,Insurance,"Foster, Rizzo, Rapp",CBRE,"Brookfield Asset Management, China Investment ...",NaN,...,0,0,0,0,0,0,0,0,0,1
3,3,243546719,None,None,None,None,None,None,"Brookfield Asset Management, China Investment ...",NaN,...,0,0,0,0,0,0,0,0,0,1
4,4,243552313,None,Neil Golmacher,Newmark Grubb Knight Frank,Finance and Insurance,"Foster, Rizzo, Rapp",CBRE,"Brookfield Asset Management, China Investment ...",NaN,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6262,6262,243550493,None,"Ades, Dempsey, Finkleman",CBRE,"Data Processing, Hosting, and Related Services","Peter Riguardi, Paul Glickman, Mitchell Konske...",JLL,Edward J. Minskoff Equities,NaN,...,0,0,0,0,0,0,0,0,0,1
6263,6263,243550496,None,"Ades, Dempsey, Finkleman",CBRE,"Data Processing, Hosting, and Related Services","Peter Riguardi, Paul Glickman, Mitchell Konser...",JLL,Edward J. Minskoff Equities,NaN,...,0,0,0,0,0,0,0,0,0,1
6264,6264,243550496,None,"Ades, Dempsey, Finkleman",CBRE,"Data Processing, Hosting, and Related Services","Peter Riguardi, Paul Glickman, Mitchell Konser...",JLL,Edward J. Minskoff Equities,NaN,...,0,0,0,0,0,0,0,0,0,1
6265,6265,243550497,None,"Ades, Dempsey, Finkleman",CBRE,"Data Processing, Hosting, and Related Services","Peter Riguardi, Paul Glickman, Mitchell Konser...",JLL,Edward J. Minskoff Equities,NaN,...,0,0,0,0,0,0,0,0,0,1


## Regressions:

In [64]:
import numpy as np
import statsmodels.api as sm
from patsy import dmatrices

For statsmodels, we set up the exogenous (independent, predictor, regressor, etc.) and endogenous (dependent, response, regressand, etc.) variables, using patsy to prepare the design matrices

#### Submarkets:

In [65]:
submarkets = [\
'submarket_Chelsea',\
'submarket_CityHall',\
'submarket_ColumbusCircle',\
'submarket_FiDi',\
'submarket_UnionSq',\
'submarket_GrandCentral',\
'submarket_HudsonSq',\
'submarket_HudsonYards',\
'submarket_MadisonFifthAve',\
'submarket_MidtownE',\
'submarket_MurrayHill',\
'submarket_GreenwichVill',\
'submarket_NorthManhattan',\
'submarket_ParkAve',\
'submarket_PennStation',\
'submarket_SixthAve',\
'submarket_SoHo',\
'submarket_TimeSq',\
'submarket_TimeSqSouth',\
'submarket_Tribeca',\
'submarket_UNPlaza',\
'submarket_UpperEast',\
'submarket_UpperWest',\
'submarket_WTC']

['submarket_Chelsea',
 'submarket_CityHall',
 'submarket_ColumbusCircle',
 'submarket_FiDi',
 'submarket_UnionSq',
 'submarket_GrandCentral',
 'submarket_HudsonSq',
 'submarket_HudsonYards',
 'submarket_MadisonFifthAve',
 'submarket_MidtownE',
 'submarket_MurrayHill',
 'submarket_GreenwichVill',
 'submarket_NorthManhattan',
 'submarket_ParkAve',
 'submarket_PennStation',
 'submarket_SixthAve',
 'submarket_SoHo',
 'submarket_TimeSq',
 'submarket_TimeSqSouth',
 'submarket_Tribeca',
 'submarket_UNPlaza',
 'submarket_UpperEast',
 'submarket_UpperWest',
 'submarket_WTC']

In [66]:
submarkets.remove('submarket_GrandCentral') # all fixed-effects relative to Grand Central
submarkets.remove('submarket_NorthManhattan') # remove submarket_NorthManhattan b/c has no observations (they are dropped when we clear the empty walkscore and transitscore observations)

In [67]:
variables = submarkets.copy()
variables.insert(0, 'log_effective_rent_usd')
table_submarkets = table[variables]

In [71]:
submarket_terms = " + ".join(submarkets)
submarket_terms = 'log_effective_rent_usd ~ ' + submarket_terms
y, X = dmatrices(submarket_terms, data=table_submarkets, return_type='dataframe')

In [77]:
model = sm.OLS(y, X)
res = model.fit()
print(res.summary())

                              OLS Regression Results                              
Dep. Variable:     log_effective_rent_usd   R-squared:                       0.043
Model:                                OLS   Adj. R-squared:                  0.039
Method:                     Least Squares   F-statistic:                     12.64
Date:                    Mon, 11 May 2020   Prob (F-statistic):           4.43e-45
Time:                            16:12:54   Log-Likelihood:                 471.92
No. Observations:                    6267   AIC:                            -897.8
Df Residuals:                        6244   BIC:                            -742.7
Df Model:                              22                                         
Covariance Type:                nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [24]:
df

,Department,Lottery,Literacy,Wealth,Region
0,Ain,41,37,73,E
1,Aisne,38,51,22,N
2,Allier,66,13,61,C
3,Basses-Alpes,80,46,76,E
4,Hautes-Alpes,79,69,83,E
...,...,...,...,...,...
80,Vendee,68,28,56,W
81,Vienne,40,25,68,W
82,Haute-Vienne,55,13,67,C
83,Vosges,14,62,82,E


In [26]:
from patsy import dmatrices

In [27]:
y, X = dmatrices('Lottery ~ Literacy + Wealth + Region', data=df, return_type='dataframe')

In [28]:
y

,Lottery
0,41.0
1,38.0
2,66.0
3,80.0
4,79.0
...,...
80,68.0
81,40.0
82,55.0
83,14.0


In [29]:
X

,Intercept,Region[T.E],Region[T.N],Region[T.S],Region[T.W],Literacy,Wealth
0,1.0,1.0,0.0,0.0,0.0,37.0,73.0
1,1.0,0.0,1.0,0.0,0.0,51.0,22.0
2,1.0,0.0,0.0,0.0,0.0,13.0,61.0
3,1.0,1.0,0.0,0.0,0.0,46.0,76.0
4,1.0,1.0,0.0,0.0,0.0,69.0,83.0
...,...,...,...,...,...,...,...
80,1.0,0.0,0.0,0.0,1.0,28.0,56.0
81,1.0,0.0,0.0,0.0,1.0,25.0,68.0
82,1.0,0.0,0.0,0.0,0.0,13.0,67.0
83,1.0,1.0,0.0,0.0,0.0,62.0,82.0
